# General Description:

For crdm

Plotting choice as a function of delay_amount, subdivded by delay_wait_time and immediate_amount

Importing libraries and mounting Google Drive

In [52]:
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import os,sys


In [53]:
# global parameters
WAIT_DICT = {'now':0,'in 1 day': 1/30.0,'in 1 week':0.25,'in 1 month': 1, 'in 3 months':3, 'in 1 year': 12, 'in 3 months': 3, 'in 5 years':60}

Defining count_tuples() function to return the unique items and their frequencies 

In [54]:
def resync_df(df,cols=[]):
    # rename columns and return pertinent columns
    df = df.rename(columns={'sure_am':'crdm_sure_amt','sure_p':'crdm_sure_p','lot_top':
                            'crdm_lott_top','lot_bot':'crdm_lott_bot','lot_p':'crdm_lott_p', 
                            'amb_lev':'crdm_amb_lev', 'domain':'crdm_domain',
                            'trialResp.keys':'crdm_trial_resp.keys', 'trialResp.corr':'crdm_choice',
                            'Rating':'crdm_conf_resp.keys'})
    df['crdm_lott_amt'] = df['crdm_lott_top'] + df['crdm_lott_bot']
    df['crdm_trial_type'] = 'task'
    # df['crdm_trial_resp.keys'] = df['crdm_choice']
    # df_len = df.shape[0]
    # df['responded'] = df['crdm_trial_resp.keys'].notna()
    # if not df['responded'].all():
    #     non_responses_nb = df['responded'].value_counts()[False]
    #     print('\n**WARNING** We dropped {0} of {1} non responses that were left blank'.format(non_responses_nb,df_len))
    #     df = df.loc[df['responded'],:].reset_index(drop=True)    
    return df[cols]
  
def tabulate_col(df,fn='/tmp/',col='crdm_immed_amt',save=False,verbose=True):
    count_df = df[col].value_counts().sort_index()
    count_df = count_df.reset_index()
    if verbose:
        print(count_df)
    #create .csv file with this info
    if save:
        print("Saving to: {}".format(fn))
        count_df.to_csv(fn)

def sync_and_save(split_dir,fn,cols=[],header=0,save=False):
    try:
        df = pd.read_csv(fn,header=header)
    except Exception as e:
        print('We got an Exception: {}'.format(e))
        print('We will move on the next spreadsheet')
        return
    subject = get_subject(df)
    subj_crdm_dir = os.path.join(split_dir,subject,'crdm')
    if not os.path.exists(subj_crdm_dir):
        print('Making subjects crdm directory : {}'.format(subj_crdm_dir))
        os.makedirs(subj_crdm_dir)
    df = resync_df(df,cols=cols)
    subj_crdm_fn = os.path.join(subj_crdm_dir,'{}_crdm.csv'.format(subject))
    print('Resynced and saving to : {}'.format(subj_crdm_fn))
    df.to_csv(subj_crdm_fn,index=0)

def get_subject(df):
    participant = int(df['participant'].mode().iat[0])
    session = int(df['session'].mode().iat[0])
    subj = 'NIDA_p{0:03d}_s{1:03d}'.format(participant,session)
    return subj


In [55]:
raw_csv_dir = '/Volumes/UCDN/datasets/NIDA/raw_csv/Data_risky_decision_noARC/'
split_dir = '/Volumes/UCDN/datasets/NIDA/split/'
cols = ['crdm_choice','crdm_sure_amt','crdm_lott_amt','crdm_sure_p','crdm_lott_p','crdm_lott_top','crdm_lott_bot',
        'crdm_amb_lev','crdm_domain','crdm_trial_type','crdm_trial_resp.keys','crdm_conf_resp.keys']
#get set of all good data files for analysis
good_files = sorted(glob(os.path.join(raw_csv_dir, '*.csv')))
if (not good_files):
    print("No good files available. Check file path.")
    sys.exit()

for i, subj_fn in enumerate(good_files):
    print('Working on file: {}'.format(subj_fn))
    sync_and_save(split_dir,subj_fn,cols=cols,header=0,save=False)

Working on file: /Volumes/UCDN/datasets/NIDA/raw_csv/Data_risky_decision_noARC/1_CDRM_fMRI_v10_2023-04-10_12h51.39.579.csv
Resynced and saving to : /Volumes/UCDN/datasets/NIDA/split/NIDA_p001_s001/crdm/NIDA_p001_s001_crdm.csv
Working on file: /Volumes/UCDN/datasets/NIDA/raw_csv/Data_risky_decision_noARC/1_CDRM_fMRI_v10_2023-04-26_14h54.36.902.csv
Resynced and saving to : /Volumes/UCDN/datasets/NIDA/split/NIDA_p001_s002/crdm/NIDA_p001_s002_crdm.csv
Working on file: /Volumes/UCDN/datasets/NIDA/raw_csv/Data_risky_decision_noARC/2_CDRM_fMRI_v10_2023-02-17_15h28.40.162.csv
Resynced and saving to : /Volumes/UCDN/datasets/NIDA/split/NIDA_p002_s001/crdm/NIDA_p002_s001_crdm.csv
Working on file: /Volumes/UCDN/datasets/NIDA/raw_csv/Data_risky_decision_noARC/2_CDRM_fMRI_v10_2023-02-24_14h11.04.431.csv
Resynced and saving to : /Volumes/UCDN/datasets/NIDA/split/NIDA_p002_s002/crdm/NIDA_p002_s002_crdm.csv
Working on file: /Volumes/UCDN/datasets/NIDA/raw_csv/Data_risky_decision_noARC/3_CDRM_fMRI_v10_2